In [1]:
!pip install tensorflow_text

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np

In [5]:
dataFrame= pd.read_json('/Users/Hp/Downloads/DL_HW7/Musical_Instruments_5.json',lines=True)

In [6]:
dataFrame.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"
5,A2A039TZMZHH9Y,B00004Y2UT,"Bill Lewey ""blewey""","[0, 0]",So good that I bought another one. Love the h...,5,The Best Cable,1356048000,"12 21, 2012"
6,A1UPZM995ZAH90,B00004Y2UT,Brian,"[0, 0]","I have used monster cables for years, and with...",5,Monster Standard 100 - 21' Instrument Cable,1390089600,"01 19, 2014"
7,AJNFQI3YR6XJ5,B00004Y2UT,"Fender Guy ""Rick""","[0, 0]",I now use this cable to run from the output of...,3,Didn't fit my 1996 Fender Strat...,1353024000,"11 16, 2012"
8,A3M1PLEYNDEYO8,B00004Y2UT,"G. Thomas ""Tom""","[0, 0]",Perfect for my Epiphone Sheraton II. Monster ...,5,Great cable,1215302400,"07 6, 2008"
9,AMNTZU1YQN1TH,B00004Y2UT,Kurt Robair,"[0, 0]",Monster makes the best cables and a lifetime w...,5,Best Instrument Cables On The Market,1389139200,"01 8, 2014"


In [7]:
dataFrame['overall'].value_counts()

5    6938
4    2084
3     772
2     250
1     217
Name: overall, dtype: int64

In [8]:
dataFrame['identifier']=dataFrame['overall'].apply(lambda x:1 if x>=4 else 0)

In [9]:
dataFrame.tail(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,identifier
10251,A3M1PLEYNDEYO8,B00JBIVXGC,"G. Thomas ""Tom""","[0, 0]",True to phosphor bronze these strings have a m...,5,"they sound great, feel great",1404172800,"07 1, 2014",1
10252,A1SD1C8XK3Z3V1,B00JBIVXGC,"guitfiddleblue ""guitfiddleblue""","[0, 0]",I've used Elixirs for about five years now. Th...,5,Elixirs just sound good,1404086400,"06 30, 2014",1
10253,A2VRAT69JDAD3W,B00JBIVXGC,"Jason Whitt ""Whittmeister""","[0, 0]","I'm a D'Addario man myself, but hey free is fr...",4,"I'm a D'Addario man myself, but hey free is ...",1405900800,"07 21, 2014",1
10254,A306NASGVUDFKF,B00JBIVXGC,"Jeffrey E ""jeffinaustintx""","[0, 0]",I really like these strings. While they are n...,5,I really like these strings,1404432000,"07 4, 2014",1
10255,A1TSKKBNV38E8Y,B00JBIVXGC,"K. Harriger ""K.R. Harriger""","[0, 0]",I have lots of friends who play these strings....,3,"Hmmm.... I like them, but with a lot of reserv...",1403654400,"06 25, 2014",0
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5,Five Stars,1405814400,"07 20, 2014",1
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014",1
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4,Good for coated.,1405987200,"07 22, 2014",1
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4,Taylor Made,1404172800,"07 1, 2014",1
10260,A2WA8TDCTGUADI,B00JBIVXGC,TheTerrorBeyond,"[0, 0]","These strings are really quite good, but I wou...",4,"These strings are really quite good, but I wou...",1405468800,"07 16, 2014",1


In [10]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

model_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [11]:
bert_preprocess_url= hub.KerasLayer(preprocess_url)

In [12]:
bert_model_url=hub.KerasLayer(model_url)

In [13]:
myinputs=tf.keras.layers.Input(shape=(),dtype=tf.string,name='InputLayer')

preprocessed_text= bert_preprocess_url(myinputs)

myoutputs=bert_model_url(preprocessed_text)['pooled_output']

myoutputs= tf.keras.layers.Dense(128,activation='tanh',name='HiddenLayer1')(myoutputs)
myoutputs= tf.keras.layers.Dense(64,activation='tanh',name='HiddenLayer2')(myoutputs)
myoutputs=tf.keras.layers.Dense(1,activation='sigmoid',name='Outputyer')(myoutputs)

model = tf.keras.Model(inputs=myinputs, outputs=myoutputs)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['InputLayer[0][0]']             
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [15]:
model.compile(loss='binary_crossentropy'
                     ,optimizer='adam'
                     ,metrics=['accuracy'])

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(dataFrame['reviewText'],dataFrame['identifier'])

In [18]:
model.fit(x_train,y_train,epochs=1)

241/241 [==============================] - 2050s 8s/step - loss: 0.3740 - accuracy: 0.8782


In [19]:
model.evaluate(x_test,y_test)

81/81 [==============================] - 675s 8s/step - loss: 0.3620 - accuracy: 0.8811


[0.3619554936885834, 0.8811379671096802]

In [20]:
check_predictions=[
    'This pop filter is great. It looks and performs like a studio filter.'
]

In [21]:
model.predict(check_predictions)

1/1 [==============================] - 4s 4s/step


array([[0.90638846]], dtype=float32)

In [22]:
model.save('/Users/Hp/Downloads/DL_HW7/AMZ/1')

INFO:tensorflow:Assets written to: /Users/Hp/Downloads/DL_HW7/AMZ/1\assets


INFO:tensorflow:Assets written to: /Users/Hp/Downloads/DL_HW7/AMZ/1\assets


In [ ]:
#Deploy the model in docker container

docker run -t --rm -p -d 8501:8501 -v "C:\Users\Hp\Downloads\DL_HW7\AMZ:/models/AMZ"  -e MODEL_NAME=AMZ  tensorflow/serving &

In [ ]:
curl -X GET http://localhost:8501/v1/models/AMZ